Vamos a pasar el .pkl a .csv para polars

In [1]:
import polars as pl
import pandas as pd

In [2]:
schema = {
    "session_id": pl.Int64,
    "date": pl.Utf8,               # String en Polars se representa como Utf8
    "timestamp_local": pl.Utf8,   # String -> Utf8
    "add_to_cart": pl.Int64,
    "user_id": pl.Float64,
    "country": pl.Int64,
    "partnumber": pl.Int32,
    "device_type": pl.Int64,
    "pagetype": pl.Float64
}

schema_clients = {
    "user_id": pl.Float64,
    "country": pl.Int64,
    "R": pl.Int64,
    "F": pl.Int64,
    "M": pl.Float64
}


In [3]:
train = pl.read_csv("../../data/raw/train.csv", schema=schema)
clients = pl.read_csv("../../data/raw/users_data.csv", schema=schema_clients)
products_pandas = pd.read_pickle("../../data/raw/products.pkl")
products = pl.from_pandas(products_pandas)

In [32]:
result_q1 = (
    products
    .filter((pl.col("color_id") == 3) & (pl.col("discount") == 1))
    .sort("family", descending=False)
    .select("partnumber")
    .head(1)
)

print("Q1 Result:", result_q1)


Q1 Result: shape: (1, 1)
┌────────────┐
│ partnumber │
│ ---        │
│ i32        │
╞════════════╡
│ 17265      │
└────────────┘


In [6]:
# Filtrar usuarios con compras totales < 500 y contar cuántos hay en cada país
filtered_clients = (
    clients
    .filter(pl.col("M") < 500)  # Filtrar por compras < 500
    .group_by("country")  # Agrupar por país
    .agg(pl.len().alias("count"))  # Contar los usuarios por país
    .sort("count", descending=True)  # Ordenar por cantidad de usuarios
    .select('country')
    .head(1)
)

# País con más usuarios con compras < 500
top_country = filtered_clients["country"]

# Filtrar usuarios en el país top y ordenar por F, R, y user_id
result_q2 = (
    clients
    .filter(pl.col("country") == top_country)  # Filtrar por el país top
    .sort(["F", "R", "user_id"], descending=[False, True, False])  # Ordenar
    .select('user_id')  # Obtener el primer usuario
    .head(1)
)

print("Q2 Result:", result_q2)

Q2 Result: shape: (1, 1)
┌──────────┐
│ user_id  │
│ ---      │
│ f64      │
╞══════════╡
│ 187374.0 │
└──────────┘


In [ ]:
# result_q3 = (
#     train
#     .filter(pl.col("add_to_cart") == 1)  # Filtrar productos añadidos al carrito
#     .group_by("partnumber")
#     .agg([
#         pl.len().alias("total_visits"),
#         pl.sum("add_to_cart").alias("cart_adds"),
#     ])
#     #.with_columns((pl.col("total_visits") / pl.col("cart_adds")).alias("visits_per_cart_add"))
#     .select(pl.col("visits_per_cart_add").mean().round(2))
# )

result_q3_filtered= (
    train
    .group_by('partnumber')  # Agrupar por 'partnumber'
    .agg(pl.sum('add_to_cart').alias('cart_adds'))  # Sumar 'add_to_cart' para obtener 'cart_adds'
    .filter(pl.col('cart_adds') > 1)  # Filtrar solo los productos con 'cart_adds' > 1
)

result_q3 = (
    train
    .filter(pl.col('partnumber').is_in(result_q3_filtered['partnumber']))
    .group_by("partnumber")
    .agg([
        pl.len().alias("total_visits"),
        pl.sum("add_to_cart").alias("cart_adds"),
    ])
    .with_columns((pl.col("total_visits") / pl.col("cart_adds")).alias("visits_per_cart_add"))
    .select(pl.col("visits_per_cart_add").mean().round(2))
)

print(result_q3) #MAL



shape: (1, 1)
┌─────────────────────┐
│ visits_per_cart_add │
│ ---                 │
│ f64                 │
╞═════════════════════╡
│ 21.9                │
└─────────────────────┘


In [18]:
import polars as pl

# Supongamos que tienes un DataFrame 'interactions' con las columnas 'partnumber', 'add_to_cart', y 'user_id'

# Filtrar interacciones donde 'add_to_cart' es 1 (producto añadido al carrito)
added_to_cart = (
    train
    .filter(pl.col("add_to_cart") == 1)  # Solo interacciones donde se añade al carrito
)

# Contar las interacciones por producto (partnumber) antes de añadir al carrito
# Para esto, debemos contar cuántas veces un producto ha sido visitado antes de que se agregue al carrito
# Primero, obtener las interacciones donde 'add_to_cart' es 1 y contar las visitas previas

product_visits = (
    train
    .filter(pl.col("add_to_cart") == 0)  # Solo interacciones donde no se agregó al carrito
    .group_by("partnumber")  # Agrupar por producto
    .agg(pl.len().alias("visits_before_add_to_cart"))  # Contar las visitas
)

# Obtener el número de interacciones de los productos que han sido añadidos al carrito
product_added = (
    added_to_cart
    .group_by("partnumber")
    .agg(pl.len().alias("added_to_cart_count"))  # Contar las veces que se añade al carrito
)

# Unir ambos DataFrames para obtener las visitas y las adiciones al carrito por producto
result = product_visits.join(product_added, on="partnumber", how="inner")

# Calcular cuántas veces de media se visitan los productos antes de ser añadidos al carrito
average_visits = result.select(
    (pl.col("visits_before_add_to_cart") / pl.col("added_to_cart_count")).alias("average_visits")
)

# Obtener el promedio general de visitas antes de añadir al carrito
avg_visits_value = average_visits.select(pl.col("average_visits").mean()).to_numpy()[0][0]

# Redondear el resultado a 2 decimales
print(f"Average visits before adding to cart: {avg_visits_value:.2f}")


Average visits before adding to cart: 22.78


In [7]:
products_mini = products.select(['partnumber', 'discount'])
train_concat = train.join(products_mini, on='partnumber', how='inner')

result_q4 = (
    train_concat
    .filter((pl.col("add_to_cart") == 1) & (pl.col("discount") == 1))
    .group_by("device_type")
    .agg(pl.len().alias("usage_count"))  # Contar el número de filas para cada dispositivo
    .sort("usage_count", descending=True)  # Ordenar por la frecuencia de uso
    .head(1)
)

print("Q4 Result:", result_q4)


Q4 Result: shape: (1, 2)
┌─────────────┬─────────────┐
│ device_type ┆ usage_count │
│ ---         ┆ ---         │
│ i64         ┆ u32         │
╞═════════════╪═════════════╡
│ 1           ┆ 169439      │
└─────────────┴─────────────┘


In [8]:
# Paso 1: Filtrar los usuarios con las frecuencias de compra más altas dentro de su país
top_3_frequency_users = (
    clients
    .sort(["country", "F"], descending=[False, True])  # Ordenar por país y frecuencia de compra
    .group_by("country")
    .head(3)  # Seleccionar los 3 usuarios con mayor frecuencia dentro de su país
)

# Paso 2: Filtrar por `device_type == 3` y contar las interacciones con productos
result_q5 = (
    train  # Supongo que 'sessions' contiene las interacciones de los usuarios
    .filter(pl.col("user_id").is_in(top_3_frequency_users["user_id"]))  # Filtrar solo los usuarios del top 3
    .filter(pl.col("device_type") == 3)  # Filtrar por `device_type == 3`
    .group_by("user_id")  # Agrupar por usuario
    .agg(pl.n_unique("partnumber").alias("unique_interactions"))  # Contar productos distintos
    .sort("unique_interactions", descending=True)  # Ordenar por interacciones más altas
    .head(1)  # Seleccionar el usuario con más interacciones
)

# Resultado final
print("Q5 Result:", result_q5)


Q5 Result: shape: (1, 2)
┌─────────┬─────────────────────┐
│ user_id ┆ unique_interactions │
│ ---     ┆ ---                 │
│ f64     ┆ u32                 │
╞═════════╪═════════════════════╡
│ 72153.0 ┆ 81                  │
└─────────┴─────────────────────┘


In [ ]:
clients_mini = (
    clients
    .select([
        pl.col('user_id'),
        pl.col('country').alias('user_country')
    ])
)
products_mini = products.select(['partnumber', 'family'])
train_mini = train.select(['partnumber', 'country'])

clients_concat = clients_mini.join(train, on='user_id', how='inner')
clients_concat = clients_concat.join(products_mini, on='partnumber', how='inner')

In [5]:
result_q6 = (
    clients_concat
    .filter(pl.col("country") != pl.col("user_country"))  # Filtrar interacciones fuera del país del usuario
    .select("family")
    .n_unique()
)

print("Q6 Result:", result_q6)

Q6 Result: 116


In [11]:
products_mini_2 = products.select(['partnumber', 'family'])
train_concat_2 = train.join(products_mini_2, on='partnumber', how='inner')
train_concat_2

session_id,date,timestamp_local,add_to_cart,user_id,country,partnumber,device_type,pagetype,family
i64,str,str,i64,f64,i64,i32,i64,f64,i32
64,"""2024-06-06""","""2024-06-06 16:43:17.389""",0,null,29,14327,1,24.0,22
117,"""2024-06-08""","""2024-06-08 15:11:02.782""",0,null,57,38422,1,24.0,73
117,"""2024-06-08""","""2024-06-08 15:11:44.797""",0,null,57,19763,1,24.0,156
579,"""2024-06-05""","""2024-06-05 19:24:48.397""",0,null,29,30253,1,24.0,53
1220,"""2024-06-04""","""2024-06-04 08:21:13.476""",0,480729.0,25,1592,1,24.0,73
…,…,…,…,…,…,…,…,…,…
5170695,"""2024-06-07""","""2024-06-07 17:57:24.644""",0,null,34,39901,3,24.0,96
5171109,"""2024-06-04""","""2024-06-04 12:34:05.430""",0,null,29,38638,1,24.0,97
5171307,"""2024-06-07""","""2024-06-07 07:50:02.549""",0,null,25,10883,1,24.0,156


In [15]:
from datetime import datetime

# Filtrar interacciones de los primeros 7 días de junio
result_q7 = (
    train_concat_2
    .filter(
        (pl.col("add_to_cart") == 1) &
        (pl.col("date").str.strptime(pl.Date, format="%Y-%m-%d") >= datetime(2024, 6, 1)) &
        (pl.col("date").str.strptime(pl.Date, format="%Y-%m-%d") <= datetime(2024, 6, 7))
    )
    .group_by(["family", "pagetype"])
    .agg([pl.len().alias('count')])
    .sort(["family", "count", "pagetype"], descending=[False, True, False])
    .group_by("family")
    .agg(pl.first("pagetype").alias("most_frequent_pagetype"))
    .to_dicts()
)
for d in result_q7:
    for k in d:
        d[k] = int(d[k])

print("Q7 Result:", result_q7)

Q7 Result: [{'family': 1, 'most_frequent_pagetype': 24}, {'family': 2, 'most_frequent_pagetype': 24}, {'family': 3, 'most_frequent_pagetype': 24}, {'family': 4, 'most_frequent_pagetype': 24}, {'family': 5, 'most_frequent_pagetype': 24}, {'family': 6, 'most_frequent_pagetype': 24}, {'family': 7, 'most_frequent_pagetype': 24}, {'family': 8, 'most_frequent_pagetype': 24}, {'family': 9, 'most_frequent_pagetype': 24}, {'family': 10, 'most_frequent_pagetype': 24}, {'family': 11, 'most_frequent_pagetype': 24}, {'family': 12, 'most_frequent_pagetype': 24}, {'family': 13, 'most_frequent_pagetype': 24}, {'family': 14, 'most_frequent_pagetype': 24}, {'family': 15, 'most_frequent_pagetype': 24}, {'family': 16, 'most_frequent_pagetype': 24}, {'family': 17, 'most_frequent_pagetype': 24}, {'family': 18, 'most_frequent_pagetype': 24}, {'family': 19, 'most_frequent_pagetype': 24}, {'family': 21, 'most_frequent_pagetype': 24}, {'family': 22, 'most_frequent_pagetype': 24}, {'family': 23, 'most_frequent_p

In [29]:
query = (
    train_concat_2
    .group_by('pagetype')
    .agg(pl.len().alias('n_pagetypes'))
    .sort('n_pagetypes', descending=True)
)
print(query)

shape: (38, 2)
┌──────────┬─────────────┐
│ pagetype ┆ n_pagetypes │
│ ---      ┆ ---         │
│ f64      ┆ u32         │
╞══════════╪═════════════╡
│ 24.0     ┆ 46256324    │
│ 8.0      ┆ 129640      │
│ 6.0      ┆ 113491      │
│ 19.0     ┆ 19534       │
│ 16.0     ┆ 15359       │
│ …        ┆ …           │
│ 33.0     ┆ 1           │
│ 27.0     ┆ 1           │
│ 4.0      ┆ 1           │
│ 30.0     ┆ 1           │
│ 35.0     ┆ 1           │
└──────────┴─────────────┘


Normal que salga todo el 24

In [31]:
import json
with open('../../predictions/example_predictions_test.json', 'w') as file:
    res = {}
    for d in result_q7:
        res[d['family']] = int(d['most_frequent_pagetype'])
    json.dump(res, file, indent=4)